####Welcome to the example notebook for FIFEforSpark! 

You may recognize the following example from FIFE's notebook found [here](https://github.com/IDA-HumanCapital/fife/blob/master/examples/country_leadership.ipynb). Like that example notebook, we use the July 2020 edition of the Rulers, Elections, and Irregular Governance dataset (REIGN) dataset, a monthly panel of national leaders and political conditions since January 1950. We load the REIGN data directly from its online archive.

First, we import the necessary packages. In this case, we import SparkFiles which is required to read the data in from the url, in addition to several fifeforspark modules.

In [0]:
import pyspark
from pyspark import SparkFiles
import fifeforspark
from fifeforspark.utils import create_example_data2
from fifeforspark.processors import PanelDataProcessor
from fifeforspark.lgb_modelers import LGBSurvivalModeler

Now that we have the necessary packages loaded, we read in the data from a url:

In [0]:
url = "https://www.dl.dropboxusercontent.com/s/3tdswu2jfgwp4xw/REIGN_2020_7.csv?dl=0"
spark.sparkContext.addFile(url)

df = spark.read.csv("file://"+SparkFiles.get("REIGN_2020_7.csv"), header=True, inferSchema= True)

The data is stored in a Spark DataFrame which is different than you may expect if you are more familiar with FIFE. Let's examine our data a bit more.

In [0]:
df.show()

+-----+-------+------+------+-----+-------+----+----+--------------+-------------+--------------------+------------+-------+-------+--------+--------------+------------+---------------+----------+-----------+-----------+----------+-------------+---------------+--------------+-------------+-------------+---------------+-------+------------+---------+---------+-------------+------+----------+--------------------+--------+-----------+
ccode|country|leader| year|month|elected| age|male|militarycareer|tenure_months| government|anticipation|ref_ant|leg_ant|exec_ant|irreg_lead_ant|election_now|election_recent|leg_recent|exec_recent|lead_recent|ref_recent|direct_recent|indirect_recent|victory_recent|defeat_recent|change_recent|nochange_recent|delayed|lastelection| loss|irregular|prev_conflict|pt_suc|pt_attempt| precip|couprisk|pctile_risk|
+-----+-------+------+------+-----+-------+----+----+--------------+-------------+--------------------+------------+-------+-------+--------+--------------+------------+---------------+----------+-----------+-----------+----------+-------------+---------------+--------------+-------------+-------------+---------------+-------+------------+---------+---------+-------------+------+----------+--------------------+--------+-----------+
 2.0| USA|Truman|1950.0| 1.0| 1.0|66.0| 1| 0.0| 58.0|Presidential Demo...| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 2.6390574| 5.327876| 7.565793| 0.0| 0.0| 0.0| -0.0690575837052633| null| null|
 2.0| USA|Truman|1950.0| 2.0| 1.0|66.0| 1| 0.0| 59.0|Presidential Demo...| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 2.7080503| 5.332719|7.5663114| 0.0| 0.0| 0.0| -0.11372068300939| null| null|
 2.0| USA|Truman|1950.0| 3.0| 1.0|66.0| 1| 0.0| 60.0|Presidential Demo...| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 2.7725887|5.3375382|7.5668287| 0.0| 0.0| 0.0| -0.108042069627093| null| null|
 2.0| USA|Truman|1950.0| 4.0| 1.0|66.0| 1| 0.0| 61.0|Presidential Demo...| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 2.8332133|5.3423343|7.5673456| 0.0| 0.0| 0.0| -0.0416001452793281| null| null|
 2.0| USA|Truman|1950.0| 5.0| 1.0|66.0| 1| 0.0| 62.0|Presidential Demo...| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 2.8903718|5.3471074|7.5678625| 0.0| 0.0| 0.0| -0.129702783937251| null| null|
 2.0| USA|Truman|1950.0| 6.0| 1.0|66.0| 1| 0.0| 63.0|Presidential Demo...| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 2.944439| 5.351858|7.5683794| 0.0| 0.0| 0.0| -0.178496151195764| null| null|
 2.0| USA|Truman|1950.0| 7.0| 1.0|66.0| 1| 0.0| 64.0|Presidential Demo...| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 2.9957323|5.3565865| 7.568896| 0.0| 0.0| 0.0| -0.042660054596682| null| null|
 2.0| USA|Truman|1950.0| 8.0| 1.0|66.0| 1| 0.0| 65.0|Presidential Demo...| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 3.0445225|5.3612924|7.5694118| 0.0| 0.0| 0.0| -0.070590356102934| null| null|
 2.0| USA|Truman|1950.0| 9.0| 1.0|66.0| 1| 0.0| 66.0|Presidential Demo...| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 3.0910425| 5.365976|7.5699277| 0.0| 0.0| 0.0| 0.0355567077070064| null| null|
 2.0| USA|Truman|1950.0| 10.0| 1.0|66.0| 1| 0.0| 67.0|Presidential Demo...| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 3.1354942| 5.370638| 7.570443| 0.0| 0.0| 0.0| -0.138817795625302| null| null|
 2.0| USA|Truman|1950.0| 11.0| 1.0|66.0| 1| 0.0| 68.0|Presidential Demo...| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 3.1780539|5.3752785|7.5709586| 1.0| 0.0| 0.0| 0.00544418163363137| null| null|
 2.0| USA|Truman|1950.0| 12.0| 1.0|66.0| 1| 

This isn't very pleasant to look at; however, the advantage of using a Spark DataFrame here (even though this could fit on one node) is that it's distributed. Let's see how many partitions we have.

In [0]:
df.rdd.getNumPartitions()

Out[4]: 8

Wow! The data is split across 8 partitions! We can always change that; however, for this example, we will leave it as 8.

Next, we make some changes to the data to prepare it for analysis

In [0]:
from pyspark.sql.functions import lit, lpad, col, concat, date_format
from pyspark.sql.types import DateType


df = df.withColumn('country-leader', concat(col('country'),lit(":"),col('leader')))
df = df.withColumn('year-month', concat(col('year').cast('integer').cast('string'),lit("-"), lpad(col('month').cast('integer').cast('string'), 2, "0"), lit("-"),lit("01")))

df = df.withColumn('year-month', df['year-month'].cast(DateType()))

cols = ['country-leader', 'year-month'] + [x for x in df.columns if x not in ["ccode", "country-leader", "leader", "year-month"]]
df = df.select(cols)
total_obs = df.count()
df = df.drop_duplicates(subset = ["country-leader", "year-month"])
n_duplicates = total_obs - df.count()
print(f"{n_duplicates} observations with a duplicated identifier pair deleted.")

7 observations with a duplicated identifier pair deleted.

Now that we have created unique identifiers for the individual and time, we pass the data through the Panel Data Processor, specifying a value of 4 for 'TEST_INTERVALS' as we want to test the last 4 periods. For the time being, we transform the time_id back to a numeric feature given constraints regarding datetime functionality.

In [0]:
test_intervals = 4
processor = PanelDataProcessor(data=df, config = {'TEST_INTERVALS': test_intervals}, shuffle_parts = 20)
processor.build_processed_data()

Time identifier column name not given; assumed to be second-leftmost column (year-month)
Individual identifier column name not given; assumed to be leftmost column (country-leader)
Out[6]: DataFrame[country-leader: string, year-month: date, country: string, year: double, month: double, elected: double, age: double, male: int, militarycareer: double, tenure_months: double, government: string, anticipation: double, ref_ant: double, leg_ant: double, exec_ant: double, irreg_lead_ant: double, election_now: double, election_recent: double, leg_recent: double, exec_recent: double, lead_recent: double, ref_recent: double, direct_recent: double, indirect_recent: double, victory_recent: double, defeat_recent: double, change_recent: double, nochange_recent: double, delayed: double, lastelection: double, loss: double, irregular: double, prev_conflict: double, pt_suc: double, pt_attempt: double, precip: double, couprisk: double, pctile_risk: double, _period: int, _predict_obs: boolean, _test: boolean, _validation: boolean, _maximum_lead: int, _spell: bigint, _duration: bigint, _event_observed: boolean]

Now, we build the model. You can pass parameters into the model that will be passed to lightgbm as well.

In [0]:
modeler = LGBSurvivalModeler(data=processor.data)
modeler.build_model(n_intervals=test_intervals)

Now we want to see how well our model performs on the test data and take a look at the predictions

In [0]:
metrics = modeler.evaluate()

In [0]:
metrics

And finally, we want to forecast future survival probabilities for country-leaders in the last period

In [0]:
forecasts = modeler.forecast()
forecasts